# Load candidates

In [3]:
%load_ext autoreload
%autoreload 2


In [4]:
CACHE = '/home/anhphantq/vdb/cache_train'
import os

import pickle
from tqdm import tqdm
import numpy as np
train_data = np.empty((100000000, 87), dtype = np.float64)
# train_data = []
num_cands = []

with open(CACHE + '/metadata', 'rb') as f:
  metadata = pickle.load(f)

num_frags = min(metadata['num_frags'], 1000)
max_session = metadata['max_session']

offset = 0
for i in tqdm(range(num_frags)):
    with open(CACHE + f'/train_data_{i}', 'rb') as f:
      data = pickle.load(f)
      num_cands.extend(data[1])
      for d in data[0]:
        train_data[offset: d.shape[0] + offset] = d 
         
        offset += d.shape[0]






100%|██████████| 192/192 [00:50<00:00,  3.77it/s]


In [5]:
train_data = train_data[:offset]

In [6]:
import numpy as np
# train_data = np.concatenate(train_data)
num_cands = np.concatenate(num_cands)
train_data.shape, num_cands.shape

((39647892, 87), (13215964,))

In [7]:
from xgb_utils import *

In [8]:
import pandas as pd 

candidates = create_data(train_data, infer = False)


In [9]:
import pandas as pd
pd.set_option('display.max_columns', candidates.shape[1])

In [10]:
print(candidates.head())


         user        item  type  cofitness_cosub  cofitness_time_decay  \
0  11280270.0  14538988.0   0.0       163.635934             91.287938   
1  11280270.0  13684483.0   0.0       135.284312             73.146202   
2  11280270.0  13044929.0   0.0       134.798171             75.272903   
3  11280270.0  13637507.0   0.0       118.021796             65.500843   
4  11280270.0  12993383.0   0.0        98.926055             52.632650   

   num_appearance  num_cosub  coclick  cocart  coorder  num_clicks  num_carts  \
0            31.0      150.0    180.0     5.0      0.0        45.0        1.0   
1            34.0      155.0    180.0     5.0      0.0       929.0       12.0   
2            30.0      145.0    180.0    10.0      0.0        58.0        9.0   
3            27.0      130.0    150.0     5.0      0.0        51.0        9.0   
4            29.0      105.0    155.0     5.0      0.0       213.0        6.0   

   num_orders  item_degree       item_pr  recent_num_clicks  recent_

# Train params

In [11]:
from xgb_utils import *


PARAMS = {
    'max_depth' : 6,
    'eta': 0.8,
    'min_child_weight': 1000,
    'num_parallel_tree': 1,
    'tree_method': 'hist',
    'max_bin': 256,
    'objective':'rank:pairwise',   
    'booster': 'dart',
}
N_SPLITS = 3
NUM_TREE = 1000
MODEL_PATH = '/home/anhphantq/xgboost'
VERSION = '12'

# Train for each type

In [12]:
from xgb_utils import get_len_group

In [13]:
import pandas as pd 
import numpy as np
tar = pd.read_parquet('/home/anhphantq/otto/splitted_data/test_labels.parquet')
tar = tar.loc[(tar['session'] <= candidates['user'].max()) & (tar['session'] >= candidates['user'].min()) ]
aids = tar.ground_truth.explode().astype('int32').rename('item')
tar = tar[['session', 'type']].rename({'session':'user'},axis=1)
tar = tar.merge(aids, left_index=True, right_index=True, how='left')
tar['type'] = tar['type'].apply(lambda x: 0 if x == 'clicks' else 1 if x == 'carts' else 2)
tar['label'] = 1
tar['item'] = tar['item'] + max_session

In [14]:
tar

,user,type,item,label
113400,11188590,0,13507107,1
113401,11188591,0,13456936,1
113402,11188591,1,14497099,1
113402,11188591,1,14136334,1
113402,11188591,1,13870063,1
...,...,...,...,...
448302,11458773,0,14272500,1
448303,11458774,0,13067478,1
448304,11458775,0,13785229,1
448305,11458776,0,13264489,1


In [15]:
import xgboost as xgb
from sklearn.model_selection import GroupKFold
import os
import matplotlib.pyplot as plt
def train(tar_, candidates_, num_cands_, t, id_type):
  
  name = f'/{t}_only_'
  try:
    shutil.rmtree(MODEL_PATH + name + VERSION)
  except:
    pass
  else:
    print('remove old version')
  
  tar = tar_[tar_['type'] == id_type].copy()
  idx = candidates_['type'] == id_type
  candidates = candidates_[idx].copy()
  
  candidates['user_type'] = candidates['user'] * 10 + candidates['type']
  tar['user_type'] = tar['user'] * 10 + tar['type']
  
  del tar['type'], tar['user']
  candidates = candidates.merge(tar,on=['user_type','item'],how='left').fillna(0)
  candidates['label'] = candidates['label'].values.astype(np.int16)
  
  
  del candidates['type'], candidates['user_type']
  
 

  skf = GroupKFold(n_splits=N_SPLITS)
  fig, axs = plt.subplots(1,3, figsize = (30, 10))
  for fold,(train_idx, valid_idx) in enumerate(skf.split(candidates, candidates['label'], groups=candidates['user'] )):

      X_train = candidates.iloc[train_idx, 2 : -4]
      y_train = candidates.iloc[train_idx, -1]
      X_valid = candidates.iloc[valid_idx, 2 : -4]
      y_valid = candidates.iloc[valid_idx, -1]

      # weight = np.where(y_train.values > 0, 5, 1)
      # weight1 = np.where(y_valid.values > 0, 5, 1)

      dtrain = xgb.DMatrix(X_train, y_train, nthread = os.cpu_count()) 
      dtrain.set_group(get_len_group(train_idx, num_cands))
      dvalid = xgb.DMatrix(X_valid, y_valid, nthread = os.cpu_count()) 
      dvalid.set_group(get_len_group(valid_idx, num_cands))
      
      xgb_parms = { 'nthread' : os.cpu_count()
      , **PARAMS}

      model = xgb.train(xgb_parms, 
          dtrain=dtrain,
          evals=[(dtrain,'train'),(dvalid,'valid')],
          num_boost_round=NUM_TREE,
          early_stopping_rounds= 15,    
          verbose_eval=5)
      try:
        os.mkdir(MODEL_PATH + name + VERSION)
      except:
        pass
      model.save_model(MODEL_PATH + name + VERSION + f'/XGB_fold{fold}.xgb')
      xgb.plot_importance(model, ax = axs[fold], max_num_features= 50)

In [16]:
for id_type, t in enumerate(['clicks', 'carts', 'orders']):
  train(tar, candidates, num_cands, t, id_type)

[0]	train-map:0.80504	valid-map:0.80342
[5]	train-map:0.80812	valid-map:0.80571
[10]	train-map:0.80992	valid-map:0.80708
[15]	train-map:0.81113	valid-map:0.80777
[20]	train-map:0.81184	valid-map:0.80813
[25]	train-map:0.81226	valid-map:0.80812
[30]	train-map:0.81290	valid-map:0.80819
[33]	train-map:0.81312	valid-map:0.80815
[0]	train-map:0.80424	valid-map:0.80505
[5]	train-map:0.80722	valid-map:0.80747
[10]	train-map:0.80920	valid-map:0.80834
[15]	train-map:0.81063	valid-map:0.80890
[20]	train-map:0.81121	valid-map:0.80942
[25]	train-map:0.81179	valid-map:0.80950
[30]	train-map:0.81205	valid-map:0.80953
[35]	train-map:0.81256	valid-map:0.80947
[38]	train-map:0.81282	valid-map:0.80951
[0]	train-map:0.80444	valid-map:0.80417
[5]	train-map:0.80781	valid-map:0.80673
[10]	train-map:0.80950	valid-map:0.80773
[15]	train-map:0.81070	valid-map:0.80840
[20]	train-map:0.81176	valid-map:0.80866
[25]	train-map:0.81217	valid-map:0.80881
[30]	train-map:0.81256	valid-map:0.80882
[35]	train-map:0.81300